In [14]:
# 在本节中，我们将介绍如何使用PyTorch更方便地实现线性回归的训练
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
from d2lzh_pytorch import *

In [15]:
# 3.2.1 生成数据集
# 我们生成与上一节中相同的数据集。其中features是训练数据特征，labels是标签
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [16]:
# 3.2.2 读取数据
# PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。
# 在每一次迭代中，我们将随机读取包含10个数据样本的小批量
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [17]:
# 这里data_iter的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.3761, -0.5668],
        [-0.3848,  2.2108],
        [ 0.8037, -1.0587],
        [ 1.0491, -0.6454],
        [ 0.1318, -0.9359],
        [ 1.2034,  1.0105],
        [ 1.3914,  0.6257],
        [ 0.4403,  1.0868],
        [-1.6306, -1.8109],
        [ 0.6389, -0.0314]]) tensor([ 6.9003, -4.0709,  9.4136,  8.4929,  7.6588,  3.1798,  4.8568,  1.3845,
         7.1093,  5.5883])


In [ ]:
# 3.2.3 定义模型
# 首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。
# 顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。
# nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。
# 在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。
# 一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。
# 下面先来看看如何用nn.Module实现一个线性回归模型

class LinearNet(nn.Module):   
    # 定义一个名为 LinearNet 的新类，继承自 PyTorch 的 nn.Module，
    # nn.Module 是 PyTorch 中所有神经网络模块的基类，提供了参数管理、设备迁移等基础功能
    def __init__(self, n_feature):
        # 定义类的初始化方法
        # 接收一个参数 n_feature，表示输入特征的维度（即有多少个特征）

        # 调用父类的初始化方法，确保nn.Module的初始化方法被正确调用，第一个参数告诉Python从LinearNet的父类中查找__init__方法
        super(LinearNet, self).__init__()  

        # 创建一个线性层（全连接层）对象，
        # nn.Linear(n_feature, 1) 表示一个从 n_feature 维到 1 维的线性变换：y = Wx + b
        # 将这个线性层保存为类的属性 self.linear
        # 这里输出维度为 1，因为线性回归预测的是一个标量值
        self.linear = nn.Linear(n_feature, 1)

    # forward 定义前向传播：
    # 在 PyTorch 中，必须实现 forward 方法来定义计算逻辑
    # 接收输入数据 x，返回模型的输出 y
    def forward(self, x):
        # 将输入 x 传入线性层进行计算
        # self.linear(x) 等价于执行 y = Wx + b 的线性变换
        # 结果保存在变量 y 中
        y = self.linear(x)
        return y

# 创建 LinearNet 类的一个实例，命名为 net
# num_inputs 是一个之前定义的变量(值为2)，表示输入特征的数量
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


事实上我们还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。

写法一

net = nn.Sequential(

          nn.Linear(num_inputs, 1)

          # 此处还可以传入其他层

          )

写法二

net = nn.Sequential()

net.add_module('linear', nn.Linear(num_inputs, 1))

#net.add_module ......

写法三

from collections import OrderedDict

net = nn.Sequential(OrderedDict([

            ('linear', nn.Linear(num_inputs, 1))

            # ......

            ]))


print(net)

print(net[0])